In [1]:
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf

In [2]:
def get_weights(number_of_features):
    if number_of_features == 1:
        return [1.0]
    elif number_of_features == 2:
        return [2./3, 1./3]
    elif number_of_features == 3:
        return [1./2, 1./3, 1./6]
    elif number_of_features == 4:
        return [2./5, 3./10, 1./5, 1./10]

In [9]:
y_true = np.array([
    [ 
        [1, 2],
        [3, 4],
        [5, 6]
    ],
    [ 
        [10, 20],
        [30, 40],
        [50, 60]
    ]    
])

y_pred = np.array([
    [ 
        [0, 0],
        [0, 0],
        [0, 0]
    ],
    [ 
        [0, 0],
        [0, 0],
        [0, 0]
    ]
    
])
print("y_true",y_true.shape)

y_true (2, 3, 2)


In [10]:
residuals = np.square(y_true - y_pred)
residuals

array([[[   1,    4],
        [   9,   16],
        [  25,   36]],

       [[ 100,  400],
        [ 900, 1600],
        [2500, 3600]]])

## Tensorflow's way

In [15]:
averaged_mse = np.average(residuals, axis=-1, weights=get_weights(y_true.shape[-1]))
averaged_mse

array([[2.00000000e+00, 1.13333333e+01, 2.86666667e+01],
       [2.00000000e+02, 1.13333333e+03, 2.86666667e+03]])

In [12]:
averaged_mse.shape

(2, 3)

In [13]:
tf.math.reduce_mean(averaged_mse, axis=None, keepdims=False, name=None)

<tf.Tensor: shape=(), dtype=float64, numpy=707.0>

## The other way around 

In [22]:
mse_on_features = np.mean(residuals, axis=1)
mse_on_features

array([[  11.66666667,   18.66666667],
       [1166.66666667, 1866.66666667]])

In [28]:
mse_on_features.shape

(2, 2)

In [27]:
averaged_mse_2 = np.average(mse_on_features, axis=-1, weights=get_weights(mse_on_features.shape[-1]))
averaged_mse_2

array([  14., 1400.])

In [29]:
averaged_mse_2.shape

(2,)

In [26]:
np.mean(averaged_mse_2)

707.0

## Tensorflow implementation

In [30]:
import tensorflow as tf

In [41]:
def get_weights(n_features):
    if n_features == 1:
        return tf.constant([1.0], dtype=tf.float32)
    elif n_features == 2:
        return tf.constant([2./3, 1./3], dtype=tf.float32)
    elif n_features == 3:
        return tf.constant([1./2, 1./3, 1./6], dtype=tf.float32)
    elif n_features == 4:
        return tf.constant([2./5, 3./10, 1./5, 1./10], dtype=tf.float32)

In [44]:
y_true = tf.constant(y_true, dtype=tf.float32)
y_pred = tf.constant(y_pred, dtype=tf.float32)

In [45]:
mse_per_sample_features = tf.math.reduce_mean(tf.math.square(y_true - y_pred), axis=1)
mse_per_sample_features

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[  11.666667,   18.666666],
       [1166.6666  , 1866.6666  ]], dtype=float32)>

In [47]:
mse_per_sample_features * get_weights(2)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[  7.777778 ,   6.2222223],
       [777.7778   , 622.2222   ]], dtype=float32)>

In [49]:
tf.math.reduce_sum(mse_per_sample_features * get_weights(2), axis=1) 

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([  14., 1400.], dtype=float32)>